In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from tensorflow.keras.datasets import mnist
from tensorflow.keras import utils
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf

In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [3]:
def place_image_on_canvas(image, width=1200, height=1200):
    """
    Place a single MNIST image at a random location on a blank canvas of given dimensions.
    """
    # Create a blank canvas
    canvas = np.zeros((height, width))

    # Image dimensions
    img_height, img_width = image.shape

    # Ensure the image fits: choose random coordinates for the top-left corner
    max_x, max_y = width - img_width, height - img_height
    x, y = np.random.randint(0, max_x), np.random.randint(0, max_y)

    # Place the image on the canvas
    canvas[y:y+img_height, x:x+img_width] = image

    return canvas, (x, y)


def create_dataset(list_img, list_labels, n_images=100):
    """
    Create a dataset of n_images of MNIST placed randomly on individual 1200x1200 canvases.
    """
    X = []
    #y = []
    coords_x = []
    coords_y = []
    labels = []
    for i in range(n_images):
        image = list_img[i]
        # Create a canvas with the image placed randomly
        canvas, (x, y) = place_image_on_canvas(image, width=128, height=128)
        canvas = canvas.reshape(128, 128, 1)
        #flat_canvas = canvas_norm.flatten()
        # Add the canvas to the dataset
        X.append(canvas)
        labels.append(list_labels[i])
        coords_x.append(x)
        coords_y.append(y)
    
    X = np.array(X)
    coords = np.array([coords_x, coords_y]).T
    #X = X.reshape(X.shape[0], 128, 128, 1)

    return X, coords, labels

In [4]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def example_test(image, label):
    feature = {
        'image': _bytes_feature(image),
        'label': _bytes_feature(label)
    }
    #print(f'feature: {feature["label"]}')
    return tf.train.Example(features=tf.train.Features(feature=feature))

def write_in_batches(data, filename, batch_size=100):
    #print(data.shape)
    record_file = filename
    with tf.io.TFRecordWriter(record_file) as writer:
        # Iterate through the dataset in batches
        for i, set in enumerate(data.take(-1)):
            images, labels = set
            batch = np.array(images)
            #batch = crop_images(batch, i)
            serialized_image = tf.io.serialize_tensor(batch).numpy()
            
            labels = np.array(labels)
            labels_unonehot = np.argmax(labels, axis=-1)
            labels_class_indices = tf.io.serialize_tensor(labels_unonehot).numpy()

            tf_example = example_test(serialized_image, labels_class_indices)
            writer.write(tf_example.SerializeToString())

In [5]:
def create_tf_dataset(x,y):
    dataset = tf.data.Dataset.from_tensor_slices((x,y))
    dataset = dataset.batch(128)
    return dataset

In [ ]:
X_train_canvas, coords, y_train = create_dataset(X_train, y_train, n_images=1000)
X_test_canvas, coords_test, y_test = create_dataset(X_test, y_test, n_images=X_test.shape[0])

In [8]:
train_dataset = create_tf_dataset(X_train_canvas, y_train)

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [ ]:
write_in_batches(train_dataset, 'train.tfrecord')